In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import anfis
import skfuzzy as fuzz
import torch

# Load the dataset
dataset = pd.read_csv('clean_dataset.csv')
X = dataset[['Glucose', 'INS', 'BMI', 'DPF', 'Age']].values
y = dataset['Outcome'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membership function parameters
mf = [
    ('gaussmf', {'mean':0.,'sigma':1.}),
    ('gaussmf', {'mean':-1.,'sigma':2.}),
    ('gaussmf', {'mean':-4.,'sigma':10.}),
    ('gaussmf', {'mean':-7.,'sigma':7.})
]

mfc = fuzz.membership.membershipfunction.MemFuncs(mf)
anf = anfis.AnfisNet('ANFIS', X_train.shape[1], mfc)
anf.print_state()

# Train the model
# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)

# Train ANFIS
optimizer = torch.optim.SGD(anf.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()  # mean-squared error loss

for epoch in range(500):
    optimizer.zero_grad()
    output = anf(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Testing the model
X_test_tensor = torch.FloatTensor(X_test)
y_pred = anf(X_test_tensor).detach().numpy()

# Binarize the output
y_pred_bin = [round(val) for val in y_pred]

# Calculate performance metrics
acc = accuracy_score(y_test, y_pred_bin)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_bin).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Accuracy: {acc}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")


Best Parameters ki and wi:  [ 35.36472793 -64.05437464]
Accuracy based on optimized parameters: 0.6708860759493671
Sensitivity based on optimized parameters: 0.7407407407407407
Specificity based on optimized parameters: 0.6346153846153846


In [18]:
import anfis
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Read the dataset
dataset = pd.read_csv('clean_dataset.csv')
X = dataset[['Glucose', 'INS', 'BMI', 'DPF', 'Age']].values
y = dataset['Outcome'].values

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define membership function
mf = [[['gaussmf', {'mean':0.,'sigma':1.}], ['gaussmf', {'mean':-1.,'sigma':2.}], ['gaussmf', {'mean':-4.,'sigma':10.}]],
       [['gaussmf', {'mean':1.,'sigma':2.}], ['gaussmf', {'mean':-1.,'sigma':1.}], ['gaussmf', {'mean':-1.,'sigma':2.}]],
       [['gaussmf', {'mean':1.,'sigma':3.}], ['gaussmf', {'mean':2.,'sigma':3.}], ['gaussmf', {'mean':-1.,'sigma':.5}]],
       [['gaussmf', {'mean':-1.,'sigma':1.}], ['gaussmf', {'mean':-1.,'sigma':3.}], ['gaussmf', {'mean':-1.,'sigma':2.}]],
       [['gaussmf', {'mean':-1.,'sigma':1.}], ['gaussmf', {'mean':-1.,'sigma':1.}], ['gaussmf', {'mean':-1.,'sigma':2.}]]]

# Initialize ANFIS model
mfc = anfis.membershipfunction.MemFuncs(mf)
anf = anfis.ANFIS(X_train, y_train, mfc)
anf.trainHybridJangOffLine(epochs=20)

# Test the model
y_pred = anf.fittedValues
y_pred_test = anfis.predict(anf, X_test)

# Binarize the output
y_pred_bin = [round(val) for val in y_pred_test]

# Calculate performance metrics
acc = accuracy_score(y_test, y_pred_bin)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_bin).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Accuracy: {acc}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")


ModuleNotFoundError: No module named 'membership'